In [1]:
import os

In [2]:
%pwd

'd:\\StyleConsult\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\StyleConsult'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    trained_model_path_in_use: Path
    base_model_path: Path
    training_data: Path
    validation_data: Path
    params_steps_per_epoch:int
    params_validation_steps:int
    params_epochs: int
    params_batch_size: int
    params_image_size: list

In [6]:
from gender_classifier.constants import *
from gender_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Training")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "Validation")

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            trained_model_path_in_use=Path(training.trained_model_path_in_use),
            base_model_path=Path(prepare_base_model.base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),
            params_epochs=params.epochs,
            params_steps_per_epoch=params.steps_per_epoch,
            params_validation_steps=params.validation_steps,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.im_shape
        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            rotation_range=30,
            shear_range=0.3,
            zoom_range=0.3
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size
        )

        train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            class_mode='binary',
            **dataflow_kwargs
        )

        self.train_generator =  train_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            class_mode='binary',
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(str(path)) 

    def train(self):
        
        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.config.params_steps_per_epoch,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            validation_steps=self.config.params_validation_steps
        )

        self.config.trained_model_path.parent.mkdir(parents=True, exist_ok=True)

        self.save_model(self.config.trained_model_path, self.model)
        self.save_model(self.config.trained_model_path_in_use, self.model)

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2024-05-23 02:06:15,376: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-23 02:06:15,378: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-23 02:06:15,379: INFO: common: created directory at: artifacts]
[2024-05-23 02:06:15,380: INFO: common: created directory at: artifacts\training]
Found 47009 images belonging to 2 classes.
Found 11649 images belonging to 2 classes.
Epoch 1/5
70/70 [==============================] - 165s 2s/step - loss: 0.7135 - acc: 0.5232 - val_loss: 0.6896 - val_acc: 0.5567
Epoch 2/5
70/70 [==============================] - 155s 2s/step - loss: 0.6776 - acc: 0.6089 - val_loss: 0.5624 - val_acc: 0.7325
Epoch 3/5
70/70 [==============================] - 149s 2s/step - loss: 0.5876 - acc: 0.7063 - val_loss: 0.5185 - val_acc: 0.7575
Epoch 4/5
70/70 [==============================] - 156s 2s/step - loss: 0.5272 - acc: 0.7560 - val_loss: 0.4703 - val_acc: 0.7871
Epoch 5/5
70/70 [==============================] - 173s 2s/step 